## Consider a building the bicubic interpolation operator faster?
The current bicubic interpolation operator works well but is not vectorized and only runs on the CPU

In [11]:
import numpy as np
import scipy
import scipy.sparse
import jax
import jax.numpy as jnp
import matplotlib.pyplot as plt
from typing import Tuple

from jaxhps.quadrature import (
    barycentric_lagrange_interpolation_matrix_2D,
    chebyshev_points,
)

from rlc_repo_link.src.data.data_transformations import (
    prep_conv_interp_2d,
    apply_interp_2d,
    # Maybe I could leverage the separability to use the 1d operators?
    prep_conv_interp_1d,
    apply_interp_1d,
)

In [2]:
# q_fn_handle = lambda x: q_gaussian_bumps(0.75 * x)
bump_center = jnp.array([0.1, 0.0])
# bump_center = jnp.array([0.0, 0.0])
amplitude = 2
sigma = 0.15
q_fn_handle = lambda x: amplitude * jnp.exp(-0.5*jnp.sum( ((x-bump_center)/sigma)**2 ,axis=-1))

In [5]:
tree_n_per_leaf = 16
tree_L = 2
tree_p = 16
tree_n = 2**tree_L * tree_n_per_leaf

print(f"L={tree_L}, N={tree_n}")

L=2, N=64


In [6]:
leaf_cheb_x = chebyshev_points(tree_p)
leaf_cheb_y = chebyshev_points(tree_p)[::-1]
leaf_to_tree = lambda xs: jnp.concatenate([
    (xs+1) * (1/2**tree_L) - 1 + (2*i/2**tree_L)
    for i in range(2**tree_L)
])
tree_cheb_x = leaf_to_tree(leaf_cheb_x)
tree_cheb_y = leaf_to_tree(leaf_cheb_y)

cell_offset = 0 # 0.5 for centered
leaf_unif_x = (cell_offset/tree_n_per_leaf)+jnp.linspace(-1, 1, tree_n_per_leaf, endpoint=False)
leaf_unif_y = (cell_offset/tree_n_per_leaf)+jnp.linspace(-1, 1, tree_n_per_leaf, endpoint=False) # [::-1]
tree_unif_x = (cell_offset/tree_n)+jnp.linspace(-1, 1, tree_n, endpoint=False)
tree_unif_y = (cell_offset/tree_n)+jnp.linspace(-1, 1, tree_n, endpoint=False)

product_grid = lambda xs, ys: jnp.array(jnp.meshgrid(xs, ys, indexing="ij")).transpose(1,2,0).reshape(-1,2)
leaf_cheb_xy = product_grid(leaf_cheb_x, leaf_cheb_y)
leaf_unif_xy = product_grid(leaf_unif_x, leaf_unif_y)
tree_unif_xy = product_grid(tree_unif_x, tree_unif_y)
tree_cheb_xy = product_grid(tree_cheb_x, tree_cheb_y)

In [172]:
%%time
tree_unif_to_cheb_x, tree_unif_to_cheb_y = prep_conv_interp_2d(
    tree_unif_x,
    tree_unif_y,
    tree_cheb_xy,
    # use a zero-valued boundary condition to mimic empty neighboring leaves
    bc_modes=("extend", "periodic"),
)

CPU times: user 15 s, sys: 3.9 s, total: 18.9 s
Wall time: 9.7 s


In [173]:
def prep_conv_interp_1d_faster(
    points: np.ndarray,
    xi: np.ndarray,
    bc_mode: str = None,
    a_neg_half: bool = True,
) -> scipy.sparse.csr_array:
    """Alternate implementation written to use some vectorization
    Prepares a sparse array to apply convolution for cubic interpolation
    Operates in a single dimension and can be applied to each dimension independently
    to work with higher-dimension data

    Assumes that the entries in `points` are sorted and evenly spaced
    Args:
        points (ndarray): original data grid points
        xi (ndarray): array of points to be sampled as a (m,)-shaped array
        bc_mode (string): how to handle the boundary conditions
            options:
                "periodic": wrap values around
                "extend": extrapolate the missing out-of-boundary values
                    using a rule for cartesian points: f(-1) = 3*f(0) - 3*f(1) + f(2)
                    (see R. Keys 1981 paper below)
                "zero": sets values outside the boundary to zero
        a_neg_half (bool): whether to use a=-1/2 for the convolution filter (otherwise use a=-3/4)
    Returns:
        conv_filter (m by n): sparse linear filter to perform
            cubic interpolation (on padded data)
            Apply to data values with `apply_interp_{1,2}d`
            Note: padding should not be needed except for the inside edge of a polar grid

    For the choice of convolution filter for cubic convolution
    See https://en.wikipedia.org/wiki/Bicubic_interpolation#Bicubic_convolution_algorithm
    and R. Keys (1981). "Cubic convolution interpolation for digital image processing".
        IEEE Transactions on Acoustics, Speech, and Signal Processing.
        29 (6): 1153–1160. Bibcode:1981ITASS..29.1153K. CiteSeerX 10.1.1.320.776.
        doi:10.1109/TASSP.1981.1163711 .
    """
    bc_mode = bc_mode.lower() if bc_mode is not None else "zero"
    # Helper variables
    periodic_mode = bc_mode == "periodic"
    extend_mode = bc_mode == "extend"
    zero_mode = bc_mode == "zero"

    if a_neg_half:
        # with a=-1/2, standard choice (seems to be the Catmull-Rom filter?)
        cubic_conv_matrix = 0.5 * np.array(
            [[0, 2, 0, 0], [-1, 0, 1, 0], [2, -5, 4, -1], [-1, 3, -3, 1]]
        )
    else:
        # with a=-3/4, computed with sympy
        # Sometimes gives lower error but has weaker theoretical properties...
        cubic_conv_matrix = 0.25 * np.array(
            [[0, 4, 0, 0], [-3, 0, 3, 0], [6, -9, 6, -3], [-3, 5, -5, 3]]
        )

    n = points.shape[0]
    min_pt = points[0]
    # interval between regularly sampled points
    itvl = (points[-1] - points[0]) / ( n - 1 )
    m = xi.shape[0]

    # Faster to build in LIL form then convert later to CSR
    interp_op = scipy.sparse.lil_array((m, n))

    js_float, xs_offset = np.divmod(xi - min_pt, itvl)
    js = js_float.astype(int)
    # print(f"js shape: {js.shape}; xs_offset shape: {xs_offset.shape}")
    pos_rel_vec = xs_offset / itvl
    monomials_mat = np.stack(
        [
            np.ones(m),
            pos_rel_vec,
            pos_rel_vec**2,
            pos_rel_vec**3
        ], axis=1
    )
    filters_local = monomials_mat @ cubic_conv_matrix

    # First handle the cases fully in bounds...
    # Identify the target points that are fully in bounds...
    tgt_idcs = np.arange(m)
    tgt_pts_in_bounds  = np.logical_and(js>=1, js<=n-3) # boolean array
    tgt_idcs_in_bounds = tgt_idcs[tgt_pts_in_bounds]    # index array (target points in bounds)
    js_idcs_in_bounds  = js[tgt_pts_in_bounds]          # index array (~relevant source points)

    # Just load the values one-by-one to avoid unholy indexing sorcery
    filters_local_in_bounds = filters_local[tgt_pts_in_bounds]
    interp_op[tgt_idcs_in_bounds, js_idcs_in_bounds-1] = filters_local_in_bounds[:, 0]
    interp_op[tgt_idcs_in_bounds, js_idcs_in_bounds+0] = filters_local_in_bounds[:, 1]
    interp_op[tgt_idcs_in_bounds, js_idcs_in_bounds+1] = filters_local_in_bounds[:, 2]
    interp_op[tgt_idcs_in_bounds, js_idcs_in_bounds+2] = filters_local_in_bounds[:, 3]

    # Handle the boundary conditions 
    tgt_pts_out_bounds  = np.logical_not(tgt_pts_in_bounds)
    tgt_idcs_out_bounds = tgt_idcs[tgt_pts_out_bounds]
    js_idcs_out_bounds  = js[tgt_pts_out_bounds]
    if periodic_mode:
        filters_local_out_bounds = filters_local[tgt_pts_out_bounds]
        interp_op[tgt_idcs_out_bounds, (js_idcs_out_bounds-1)%n] = filters_local_out_bounds[:, 0]
        interp_op[tgt_idcs_out_bounds, (js_idcs_out_bounds+0)%n] = filters_local_out_bounds[:, 1]
        interp_op[tgt_idcs_out_bounds, (js_idcs_out_bounds+1)%n] = filters_local_out_bounds[:, 2]
        interp_op[tgt_idcs_out_bounds, (js_idcs_out_bounds+2)%n] = filters_local_out_bounds[:, 3]
    else:
        for i in tgt_idcs_out_bounds:
            x = xi[i]
            j = js[i]
            filter_local = filters_local[i]
            # Assumes zero value beyond the extra single-cell padding
            # Extrapolation rule was linear anyway so fold down the extrapolation
            # into a reduced-length filter
            if extend_mode:
                if j < 1 and (j + 3) >= 0:
                    filter_folded = filter_local[1:] + filter_local[0] * np.array(
                        [3, -3, 1]
                    )
                    interp_op[i, : j + 3] = filter_folded[: j + 3]
                elif j < n and (j + 3) >= n:
                    filter_folded = filter_local[:-1] + filter_local[-1] * np.array(
                        [1, -3, 3]
                    )
                    interp_op[i, j - 1 :] = filter_folded[: n - j + 1]
            else:  # bc_mode == "zero" case
                if j < 1 and (j + 3) >= 0:
                    interp_op[i, : j + 3] = filter_local[: j + 3]
                elif j < n and (j + 3) >= n:
                    interp_op[i, j - 1 :] = filter_local[: n - j + 1]
    # tgt_pts_out_bounds  = np.logical_not(tgt_pts_in_bounds)
    # tgt_idcs_out_bounds = tgt_idcs[tgt_pts_out_bounds]
    # for i in tgt_idcs_out_bounds:
    #     x = xi[i]
    #     j = js[i]
    #     filter_local = filters_local[i]
    #     if periodic_mode:
    #         j_idcs = np.arange(j-1, j+3) % n
    #         interp_op[i, j_idcs] = filter_local
    #     elif extend_mode:
    #         # Assumes zero value beyond the extra single-cell padding
    #         # Extrapolation rule was linear anyway so fold down the extrapolation
    #         # into a reduced-length filter
    #         if j < 1 and (j + 3) >= 0:
    #             filter_folded = filter_local[1:] + filter_local[0] * np.array(
    #                 [3, -3, 1]
    #             )
    #             interp_op[i, : j + 3] = filter_folded[: j + 3]
    #         elif j < n and (j + 3) >= n:
    #             filter_folded = filter_local[:-1] + filter_local[-1] * np.array(
    #                 [1, -3, 3]
    #             )
    #             interp_op[i, j - 1 :] = filter_folded[: n - j + 1]
    #     else:  # bc_mode == "zero" case
    #         if j < 1 and (j + 3) >= 0:
    #             interp_op[i, : j + 3] = filter_local[: j + 3]
    #         elif j < n and (j + 3) >= n:
    #             interp_op[i, j - 1 :] = filter_local[: n - j + 1]

    # Convert for slightly faster application
    interp_op = scipy.sparse.csr_array(interp_op)
    return interp_op
        
def prep_conv_interp_2d_faster(
    points_x: np.ndarray,
    points_y: np.ndarray,
    xi: np.ndarray,
    bc_modes: None | str | Tuple = None,
    a_neg_half: bool = True,
) -> Tuple[scipy.sparse.csr_array, scipy.sparse.csr_array]:
    should_split = hasattr(bc_modes, "__len__") and len(bc_modes) == 2
    bc_mode_x = bc_modes[0] if should_split else bc_modes
    bc_mode_y = bc_modes[1] if should_split else bc_modes
    interp_op_x = prep_conv_interp_1d_faster(
        points_x, xi[:, 0], bc_mode=bc_mode_x, a_neg_half=a_neg_half
    )
    interp_op_y = prep_conv_interp_1d_faster(
        points_y, xi[:, 1], bc_mode=bc_mode_y, a_neg_half=a_neg_half
    )
    return interp_op_x, interp_op_y

# def apply_interp_1d(
#     interp_op: scipy.sparse.csr_array, data_vals: np.ndarray
# ) -> np.ndarray:
#     """Applies the x/y convolutional filter operators onto the padded values
#         ~ diag(conv_x (vals) conv_y^T)
#     interp_op is   m x nx
#     data_vals is nx x {1, ny}
#     result is m (or m x ny) interpolated values
#     """
#     res = interp_op @ data_vals  # .sum(1) # (m, ny+2) to (m,)
#     return res


# def apply_interp_2d(
#     interp_op_x: scipy.sparse.csr_array,
#     interp_op_y: scipy.sparse.csr_array,
#     data_vals: np.ndarray,
# ) -> np.ndarray:
#     """Applies the x/y convolutional filters onto the padded values
#         ~ diag(conv_x (vals) conv_y^T)
#     Args:
#         interp_op_x (sparse csr array): x-dim interpolation operator with shape (m, nx)
#         interp_op_y (sparse csr array): y-dim interpolation operator with shape (m, ny)
#         data_vals (ndarray): data values arranged in a grid (nx, ny)
#     Returns:
#         result (ndarray) is the interpolated values in a (m,) array
#     """
#     post_x = interp_op_x @ data_vals  # m x (ny+2)
#     res = (post_x * interp_op_y).sum(1)
#     return res

In [174]:
%%time
tree_unif_to_cheb_x_faster, tree_unif_to_cheb_y_faster = prep_conv_interp_2d_faster(
    tree_unif_x,
    tree_unif_y,
    tree_cheb_xy,
    # use a zero-valued boundary condition to mimic empty neighboring leaves
    bc_modes=("extend", "periodic"),
)

js shape: (4096,); xs_offset shape: (4096,)
monomials_mat shape: (4096, 4)
filters_local shape: (4096, 4)
js shape: (4096,); xs_offset shape: (4096,)
monomials_mat shape: (4096, 4)
filters_local shape: (4096, 4)
CPU times: user 706 ms, sys: 95.3 ms, total: 801 ms
Wall time: 214 ms


In [175]:
matches_in_x = np.allclose(tree_unif_to_cheb_x_faster.todense(), tree_unif_to_cheb_x.todense())
matches_in_y = np.allclose(tree_unif_to_cheb_y_faster.todense(), tree_unif_to_cheb_y.todense())
print(f"Matches? {matches_in_x and matches_in_y}")
print(f"Matches in x: {matches_in_x}")
print(f"Matches in y: {matches_in_y}")

Matches? True
Matches in x: True
Matches in y: True
